# Explore MovieLens Dataset

This example leverages Mars to explore on MovieLens dataset.

## Data preparation

Download [MovieLens 25M Dataset](https://grouplens.org/datasets/movielens/25m/) first.

In [ ]:
import os

if not os.path.exists('datasets/ml-25m.zip'):
    # retrieve dataset
    !wget -O datasets/ml-25m.zip http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !unzip datasets/ml-25m.zip -d datasets

## Install libraries

Besides Mars, install libraries that are necessary for this example.

In [ ]:
!pip install pymars>=0.6.0
!pip install wordcloud pandas_bokeh

## Begin the tour of exploration with Mars

First, import all modules that we need.

In [ ]:
import warnings

import mars.dataframe as md
import numpy as np
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

pandas_bokeh.output_notebook()
warnings.filterwarnings('ignore')
MAX_WORDS = 50

### Read CSV Data into Mars DataFrame

In [ ]:
movies = md.read_csv('datasets/ml-25m/movies.csv')
ratings = md.read_csv('datasets/ml-25m/ratings.csv')
tags = md.read_csv('datasets/ml-25m/tags.csv')

In [ ]:
movies.head().execute()

In [ ]:
ratings.head().execute()

In [ ]:
tags.head().execute()

### Analyze according to release date

Extract release year from title.

In [ ]:
def extract_year(title):
    try:
        year = title.strip()[-5: -1]
        if len(year) != 4:
            return 0
        year = int(year)
        if year < 1900:
            return 0
        return year
    except:
        return 0

movies_with_date = movies.copy()
movies_with_date['release_year'] = movies.title.map(extract_year, dtype=int)
movies_with_date = movies_with_date[movies_with_date['release_year'] > 0]
movies_with_date.execute()

Plot data using `pandas_bokeh` backend to show an interactive chart.

In [ ]:
year_movies = movies_with_date.groupby('release_year').size()
year_movies.plot(backend='pandas_bokeh')

### Top 100 rating movies

In [ ]:
movie_ratings = ratings.groupby('movieId', as_index=False).agg({'rating': ['count', 'mean']})
movie_ratings.columns = ['movieId', 'count', 'avg_rating']
movie_ratings.execute()

Filter movies that has too few ratings.

In [ ]:
movie_ratings = movie_ratings[movie_ratings['count'] > 50]
has_rating_movies = movie_ratings.merge(movies)
has_rating_movies = has_rating_movies.sort_values('avg_rating', ascending=False)
has_rating_movies.head(100).execute()

## Word clouds based on tags data

In [ ]:
def extract_words(tag):
    if not isinstance(tag, str) and np.isnan(tag):
        return []
    words = tag.split()
    return [w for w in words if w not in STOPWORDS]

words = tags.tag.apply(extract_words).explode()
top_words = words.value_counts()[:MAX_WORDS].reset_index()
selected_words = words[words.isin(top_words.iloc[:, 0])]
selected_words.execute()

In [ ]:
words_str = ' '.join([t for t in selected_words.to_pandas()])
wordcloud = WordCloud(
    collocations=False, max_words=MAX_WORDS).generate(words_str)

fig = plt.figure()
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

Extract release year from title.

In [ ]:
def extract_year(title):
    try:
        year = title.strip()[-5: -1]
        if len(year) != 4:
            return 0
        year = int(year)
        if year < 1900:
            return 0
        return year
    except:
        return 0

movies_with_date = movies.copy()
movies_with_date['release_year'] = movies.title.map(extract_year, dtype=int)
movies_with_date = movies_with_date[movies_with_date['release_year'] > 0]
movies_with_date.execute()

Plot data using `pandas_bokeh` backend to show an interactive chart.

In [ ]:
year_movies = movies_with_date.groupby('release_year').size()
year_movies.plot(backend='pandas_bokeh')

### Top 100 rating movies

In [ ]:
movie_ratings = ratings.groupby('movieId', as_index=False).agg({'rating': ['count', 'mean']})
movie_ratings.columns = ['movieId', 'count', 'avg_rating']
movie_ratings.execute()

Filter movies that has too few ratings.

In [ ]:
movie_ratings = movie_ratings[movie_ratings['count'] > 50]
has_rating_movies = movie_ratings.merge(movies)
has_rating_movies = has_rating_movies.sort_values('avg_rating', ascending=False)
has_rating_movies.head(100).execute()

## Word clouds based on tags data

In [ ]:
def extract_words(tag):
    if not isinstance(tag, str) and np.isnan(tag):
        return []
    words = tag.split()
    return [w for w in words if w not in STOPWORDS]

words = tags.tag.apply(extract_words).explode()
top_words = words.value_counts()[:MAX_WORDS].reset_index()
selected_words = words[words.isin(top_words.iloc[:, 0])]
selected_words.execute()

In [ ]:
words_str = ' '.join([t for t in selected_words.to_pandas()])
wordcloud = WordCloud(
    collocations=False, max_words=MAX_WORDS).generate(words_str)

fig = plt.figure()
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()